# Querying and Managing Accounts in VAM Client
This notebook demonstrates how to:
- Perform CRUD (Create, Read, Update, Delete) operations on accounts.
- Filter accounts based on different execution venues 
- Understand the differences in querying `Account`


In [2]:
import os
from pathlib import Path

# Save the original working directory (only once)
try:
    original_wd
except NameError:
    original_wd = os.getcwd()

# Compute the target directory: two levels up from the original working directory
# For example, if original_wd is /Users/username/project/notebooks,
# then target_dir becomes /Users/username
target_dir = Path(original_wd).parents[2]

# Change the working directory to the target directory
os.chdir(target_dir)
print("Working directory set to:", os.getcwd())

import dotenv
dotenv.load_dotenv('.env')

# Import necessary modules
import pandas as pd
from mainsequence.client import Account
from mainsequence.client.utils import MARKETS_CONSTANTS


Working directory set to: /Users/jose/code/mainsequence-sdk


PydanticSchemaGenerationError: Unable to generate pydantic-core schema for [<class 'str'>]. Set `arbitrary_types_allowed=True` in the model_config to ignore this error or implement `__get_pydantic_core_schema__` on your type to fully support it.

If you got this error by calling handler(<some type>) within `__get_pydantic_core_schema__` then you likely need to call `handler.generate_schema(<some type>)` since we do not call `__get_pydantic_core_schema__` on `<some type>` otherwise to avoid infinite recursion.

For further information visit https://errors.pydantic.dev/2.11/u/schema-for-unknown-type

In [ ]:
!pip install matplotlib

## Retrieving All Active Accounts
We fetch all active accounts from the database.

### The Main Sequence Execution Venue and Main Sequence Account
The mapping between accounts and execution venues defines which portfolios can be added to specific accounts via the portfolio property `required_venues`. This restriction ensures that an account can only follow portfolios matching the defined `required_venues`. However, as a researcher, you might want to create accounts capable of tracking portfolios across multiple different venues. In such cases, you can utilize accounts within the **Main Sequence ExecutionVenue**, which allows you to mix assets and portfolios from any venue.

In [ ]:
# Query all active accounts
accounts= Account.filter(account_is_active=True)
print(f"Total active accounts: {len(accounts)}")

# Display general account information
for acc in accounts:
    print(f"Account Name: {acc.account_name}, Venue: {acc.execution_venue}, Latest Holdings: {acc.latest_holdings.holdings_date}")

# Account Historical Holdings & Risk Factors
We can get historical information as snapshot of our Accounts

In [ ]:
import datetime
import matplotlib.pyplot as plt
from mainsequence.client import AccountHistoricalHoldings,AccountRiskFactors


end_date = datetime.datetime.utcnow()
start_date = end_date - datetime.timedelta(hours=1)

# Query with a date range filter
historical_holdings= AccountHistoricalHoldings.filter(
    holdings_date__gte=start_date,
    holdings_date__lte=end_date,
    related_account__id=accounts[0].id
)


# Extract NAV values and dates from historical holdings
nav_values = [holding.nav for holding in historical_holdings if holding.nav is not None]
dates = [holding.holdings_date for holding in historical_holdings if holding.nav is not None]

# Sort data by date
sorted_data = sorted(zip(dates, nav_values), key=lambda x: x[0])

if len(sorted_data) ==0:
    raise Exception("No data found")

dates, nav_values = zip(*sorted_data)

# Plot NAV over time
plt.figure(figsize=(10, 5))
plt.plot(dates, nav_values, marker='o', linestyle='-', label="NAV Over Time")
plt.xlabel("Date")
plt.ylabel("NAV")
plt.title("Account NAV Over Time")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()


In [ ]:
risk_factors=AccountRiskFactors.filter(related_holdings__id__in=[h.id for h in historical_holdings])


In [ ]:

holdings_date_map = {h.id: h.holdings_date for h in historical_holdings}

# Prepare lists for plotting
dates = []
balances = []

# Loop over your risk factors data (a list of dictionaries)
for rf in risk_factors:
    holding_id = rf.related_holdings.id
    # Check if we have a corresponding holding date
    if holding_id in holdings_date_map.keys():
        dates.append(holdings_date_map[holding_id])
        balances.append(rf.account_balance)

# Optionally, sort the data by date if not already sorted
data = sorted(zip(dates, balances), key=lambda x: x[0])
sorted_dates, sorted_balances = zip(*data) if data else ([], [])

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(sorted_dates, sorted_balances, marker='o', linestyle='-', color='b')
plt.xlabel("Holdings Date")
plt.ylabel(" account_balance")
plt.title("Total Account Balance over Time")
plt.grid(True)
plt.show()


In [ ]:
accounts[0].uuid